### Evaluating the m5 limits ###

Create values to be used in `rubin_sim.utils.sysEngVals`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import syseng_throughputs as st

In [2]:
pd.set_option('display.precision', 3)

In [3]:
# Read bandpass information.
defaultDirs = st.setDefaultDirs()
addLosses = True
# Use the X=1.0 atmosphere!
atmosphere = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = st.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere)

/Users/lynnej/lsst_repos/rubin_sim/rubin_sim/phot_utils/bandpass.py:67: UserWarning: Wavelength sampling of 1.0 nm is > 0.2 nm, this may not work well with a Sed object. Consider resampling with resample_bandpass method.
  warnings.warn(
/Users/lynnej/lsst_repos/rubin_sim/rubin_sim/phot_utils/bandpass.py:67: UserWarning: Wavelength sampling of 800.0 nm is > 0.2 nm, this may not work well with a Sed object. Consider resampling with resample_bandpass method.
  warnings.warn(


In [4]:
# --note these are 1x30s and the overview paper uses 2x15s!!! 
# (this is used as the 'base exposure time' in sysEngVals and the m5 calculation from Cm/dCm_infinity)
# Note also this is using fiducial (old) values of the seeing to set the m5 values as well. 
# thus the resulting values are different than those reported in pstn-054, where the Cm/dCm reported values are 
# for the median delivered seeing in the opsim databases.

m5_std = st.makeM5(hardware, system, readnoise=8.8, othernoise=0, darkcurrent=0.2)

m5_std

,nexp,exptime,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,1.0,30.0,0.92,0.808,22.991,64.018,26.814,0.030,0.047,0.495,0.038,23.217,0.436,0.265,23.915,433.114,23.9,23.4
g,2.0,15.0,0.87,0.767,22.264,423.326,28.417,0.131,0.160,0.215,0.039,24.451,0.172,0.091,24.847,803.955,25.0,24.6
r,2.0,15.0,0.83,0.734,21.199,844.831,28.192,0.106,0.120,0.127,0.039,24.466,0.093,0.046,24.381,1003.147,24.7,24.3
i,2.0,15.0,0.80,0.710,20.474,1248.344,27.921,0.083,0.091,0.096,0.039,24.377,0.065,0.031,23.969,1142.755,24.0,23.6
z,2.0,15.0,0.78,0.693,19.595,1890.565,27.519,0.057,0.061,0.069,0.039,24.213,0.044,0.020,23.393,1342.078,23.3,22.9
y,2.0,15.0,0.76,0.677,18.597,2410.428,26.685,0.027,0.031,0.169,0.039,23.756,0.035,0.016,22.465,1462.863,22.1,21.7


In [5]:
# Write out values of dictionaries that can be pasted into sims_utils
output_cols = ['Zp_t', 'Tb', 'gamma', 'kAtm', 'Cm', 'dCm_infinity', 'dCm_double', 'skyMag']
for column in output_cols:
    result = '{'
    for filtername, val in zip(m5_std.index, m5_std[column]):
        result += '"%s": %f, ' % (filtername, val)
    result += '}'
    result = result.replace(', }', '}')
    result = 'self.'+column+ ' = ' + result
    print(result)

self.Zp_t = {"u": 26.813810, "g": 28.417251, "r": 28.191548, "i": 27.921078, "z": 27.519404, "y": 26.685422}
self.Tb = {"u": 0.029936, "g": 0.131088, "r": 0.106484, "i": 0.083003, "z": 0.057336, "y": 0.026597}
self.gamma = {"u": 0.037691, "g": 0.038756, "r": 0.039003, "i": 0.039125, "z": 0.039255, "y": 0.039316}
self.kAtm = {"u": 0.494560, "g": 0.215218, "r": 0.126712, "i": 0.096044, "z": 0.069145, "y": 0.168572}
self.Cm = {"u": 23.216549, "g": 24.451160, "r": 24.466245, "i": 24.376858, "z": 24.212743, "y": 23.756149}
self.dCm_infinity = {"u": 0.436258, "g": 0.171867, "r": 0.093382, "i": 0.065057, "z": 0.043910, "y": 0.034768}
self.dCm_double = {"u": 0.265201, "g": 0.090540, "r": 0.046267, "i": 0.031100, "z": 0.020171, "y": 0.015528}
self.skyMag = {"u": 22.990565, "g": 22.263735, "r": 21.199289, "i": 20.474239, "z": 19.595023, "y": 18.596712}
